# Fashion MNIST Data Science Challenge: Neural Networks and Deep Learning


This file contains the code as well as the explanations to our final model.

## Imports

In [1]:
#basic packages
import typing
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import datetime
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,BatchNormalization
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.data import Dataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

np.random.RandomState(12345)  # Set random state for constant results
tf.random.set_seed(12345)

In [2]:
# When true explanatory graphs and tensorboard are rendered -> impacts runtime
SHOW_EXPLANATORY_GRAPHS = True

In [3]:
# Clean up data set cache
CACHE_DIR = "cache"
VALIDATION_CACHE_FILE = os.path.join(CACHE_DIR, "validation_data_cache.tfcache")
TRAINING_CACHE_FILE = os.path.join(CACHE_DIR, "training_data_cache.tfcache")

if os.path.isdir(CACHE_DIR):
    # delete dir
    shutil.rmtree(CACHE_DIR)

os.mkdir(CACHE_DIR)

if os.path.isfile(VALIDATION_CACHE_FILE):
    os.remove(VALIDATION_CACHE_FILE)
if os.path.isfile(TRAINING_CACHE_FILE):
    os.remove(TRAINING_CACHE_FILE)

## Data Preparation and Augmentation

  <span style="color:red">*TODO BEWEISE + Methoden*</span>.

### Random erasing
We are using a third-party github repository that implements random_erasing. You can find it [here](https://github.com/yu4u/cutout-random-erasing).

In [5]:
# Import random eraser from its python file
%run random_eraser/random_eraser

### Image rotation and flipping

In [6]:
#Image rotation was used for data augmentation. The function names are self-explanatory
def flip_image_vertical(image: np.ndarray) -> np.ndarray:      
    return np.flip(image, axis=1)

def flip_image_horizontal(image: np.ndarray) -> np.ndarray:
    return np.flip(image, axis=0)

def rotate_270(image: np.ndarray) -> np.ndarray:
    return np.rot90(image, -1)

def rotate_90(image: np.ndarray) -> np.ndarray:
    return np.rot90(image, 1)

In [7]:
def augment_images(input_data: np.ndarray)->typing.Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    flipped_images_vertical = np.empty(shape=input_data.shape)
    flipped_images_horizontal = np.empty(shape=input_data.shape)
    rot_images_90 = np.empty(shape=input_data.shape)
    rot_images_270 = np.empty(shape=input_data.shape)
    
    for ind, im in enumerate(input_data):
        flipped_images_vertical[ind] = flip_image_vertical(im)
        flipped_images_horizontal[ind] = flip_image_horizontal(im)
        rot_images_90[ind] = rotate_90(im)
        rot_images_270[ind] = rotate_270(im)
        
    return flipped_images_vertical, flipped_images_horizontal, rot_images_90, rot_images_270

In [8]:
# About 50% Chance that any augmentation happens
ROTATION_CHANCE = 0.1
FLIP_CHANCE = 0.2 # actually 0.1 since tf.image.random_flip_left_right has 0.5 chance
CONSTRAST_CHANCE = 0.2 # actually 0.1 since tf.image.random_contrast has 0.5 chance
BRIGHTNESS_CHANCE = 0.2 # actually 0.1 since tf.image.random_brightness has 0.5 chance
ERASER_CHANCE = 0.25

In [9]:
# function that takes an image as input and creates a randomly augmented image as output
def dataset_aumentation(image: tf.Tensor, label)->np.ndarray:

    output_image = image
    
    ######## ROTATE ########
    augmentation_chance = np.random.random_sample(size=None)
    rotation_amount = np.random.randint(0, high=4)
    if (augmentation_chance <= ROTATION_CHANCE):
        # rotation_amount to degrees:
        # 0 = 0°
        # 1 = 90°
        # 2 = 180°
        # 3 = 270°
        # 4 (not possible) = 360° = 0°
        output_image = tf.image.rot90(output_image, k=rotation_amount)
    
    ######## FLIP ########
    augmentation_chance = np.random.random_sample(size=None)
    if (augmentation_chance <= FLIP_CHANCE):
        output_image = tf.image.random_flip_left_right(output_image, seed=12345)
        
    ######## CONTRAST ########
    augmentation_chance = np.random.random_sample(size=None)
    if (augmentation_chance <= CONSTRAST_CHANCE):
        output_image = tf.image.random_contrast(output_image, 0.2, 0.5, seed=12345) 
        
    ######## BRIGHTNESS ########
    augmentation_chance = np.random.random_sample(size=None)
    if (augmentation_chance <= BRIGHTNESS_CHANCE):
        output_image = tf.image.random_brightness(output_image, 0.2, seed=12345) 
        
    ######## ERASER ########
    # Uses ndim so convert to numpy array
    # Does not work yet

#     if image is not None:
#         conv_tensor = image.eval(session=tf.compat.v1.Session())
#         output_image =  get_random_eraser(p=ERASER_CHANCE, s_l=0.04, s_h=0.2, v_l=-1, v_h=1, pixel_level=True)(conv_tensor)
        
    return output_image, label

### Data Preparation

In [11]:
#1. Get the file
data_train = pd.read_csv('train.csv')
data_validate = pd.read_csv('test.csv')
data_train = np.array(data_train, dtype = 'float32') # Damit Input Daten von Keras akzeptiert werden müssen wir sie in ein Array umwandeln 
data_validate = np.array(data_validate, dtype='float32') 

In [12]:
# Since we are using a Normalization layer and a Dataset, the preprocessing is only 
# reshaping the arrays and splitting the sets
x_train = data_train[:,1:]
y_train = data_train[:,0] #label data

data_submission = data_validate

In [13]:
#reshape the array containing the images (28px x 28px and 1 channel)
image_rows = 28
image_cols = 28
image_shape = (image_rows,image_cols,1)# 1 da schwarz weiß, bei Farbbildern 3 (r,g,b)

x_train = x_train.reshape(x_train.shape[0],*image_shape)
data_submission = data_submission.reshape(data_submission.shape[0],*image_shape)

In [14]:
#split train data in train and validation set
x_train2,x_validate2,y_train2,y_validate2 = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)

In [15]:
REPEAT_AMOUNT = 2  # How many times the data set should be repeated in one epoch
# random eraser cannot be used inside Dataset since the tensor needs to be transformed to a numpy array which does not 
# work inside Dataset.map since the code is not executed eagerly inside
# Also evaluating the tensor does not work since map gives a Placeholder instead of the values
# also make_ndarry does not work for unknown reasons (bug in tensorflow or sth)
# so the data has to prepared manually
# since the data is prepared manually the repeating is done here instead of at Dataset.repeat

x_train2 = np.concatenate([x_train2 for _ in range(REPEAT_AMOUNT)])
y_train2 = np.concatenate([y_train2 for _ in range(REPEAT_AMOUNT)])

eraser = get_random_eraser(p=ERASER_CHANCE, s_l=0.04, s_h=0.2, v_l=-1, v_h=1, pixel_level=True)
x_train2 = np.array([eraser(im) for im in x_train2])

x_validate2 = np.concatenate([x_validate2 for _ in range(REPEAT_AMOUNT)])
y_validate2 = np.concatenate([y_validate2 for _ in range(REPEAT_AMOUNT)])

eraser = get_random_eraser(p=ERASER_CHANCE, s_l=0.04, s_h=0.2, v_l=-1, v_h=1, pixel_level=True)
x_validate2 = np.array([eraser(im) for im in x_validate2])

## Modeling

### Parameters

Mostly we decided our parameters by doing hyperparameter tuning via trial and error.

In [14]:
#@felix dein code hier
NUM_EPOCHS = 200
INIT_LR = 1e-3
BATCH_SIZE = 128
L2_PENALTY = 0.003

#### Optimizer
There are multiple articles mentioning SGD as the best optimizer in the long term.
[This article](https://shaoanlu.wordpress.com/2017/05/29/sgd-all-which-one-is-the-best-optimizer-dogs-vs-cats-toy-experiment/) compares the Adam and SGD optimizer and concludes that SGD with momentum and Nesterov results in a better validation accuracy.
Other articles suggest that even though Adam is faster in the beginning but SGD has better convergence in the long run.

Even though SGD might take more epochs to train, the network's accuracy ends up to be more stable.

In [15]:
our_optimizer = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR/max(NUM_EPOCHS, 1))  # max to prevent divide by 0

#### Loss Function
  <span style="color:red">*Explanation*</span>.
  The aim of this chapter is to evaluate the best loss function on an architecture. We do this by taking a subset of the training. Then we train our models with the different loss functions. Note that all loss functions are trained with the same data. After the training we evaluate the accuracy with: 1. training sample and 2. validation sample for our loss functions. To ensure that we are statistically correct we repeat this `n_survey` times. We aggregate by taking the mean of all  `n_survey` as well as the variance.

In [ ]:
# set random seed for dropouts
tf.random.set_seed(53124)

# set params and seed to reproduce code
np.random.seed(12498)
n_epoch = 4
n_survey = 6
n_models = 3
cnt = 0
# initialize accuracy and its variance
acc = np.zeros((n_epoch,n_models))
acc_sq = np.zeros((n_epoch,n_models))
# initialize validation accuracy and its variance
vacc = np.zeros((n_epoch,n_models))
vacc_sq = np.zeros((n_epoch,n_models))

for ii in np.random.randint(1,100000,n_survey):
    # split rain data in train and validation set, use different random states
    x_tr,x_val,y_tr,y_val = train_test_split(x_train2,y_train2,test_size = 0.2,random_state = ii)
    # update count
    print('__________ ',np.round(cnt/n_survey *100),'% __________')
    cnt += 1
    
    
    # iterate through different models
    for jj in range(n_models):
        if jj==0:
            ## sparse categorical crossentropy
            # simple baseline model (to beat)
            msimple = Sequential([
                Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu',input_shape=(28,28,1)),
                Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu'),
                Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'),
                Flatten(),
                Dense(256, activation='relu'),
                Dropout(.3),
                Dense(128, activation='relu'),
                Dropout(.3),
                Dense(10,activation='softmax')
            ])
            
            # compile model
            msimple.compile(optimizer='adam',
                            loss='sparse_categorical_crossentropy',
                            metrics=['accuracy'])
            
            # train model
            wsimple = msimple.fit(x_tr,y_tr,epochs=n_epoch,verbose=0,
                                  batch_size=64,
                                  validation_data=(x_validate,y_validate))
            
            print('\n sparse categorical crossentropy: ',end='')
            
            
        elif jj==1:
            ## categorical crossentropy
            # simple baseline model (to beat)
            msimple = Sequential([
                Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu',input_shape=(28,28,1)),
                Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu'),
                Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'),
                Flatten(),
                Dense(256, activation='relu'),
                Dropout(.3),
                Dense(128, activation='relu'),
                Dropout(.3),
                Dense(10,activation='softmax')
            ])
            
            # compile model
            msimple.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['accuracy'])
            
            # train model
            wsimple = msimple.fit(x_tr,to_categorical(y_tr),epochs=n_epoch,verbose=0,
                                  batch_size=64,
                                  validation_data=(x_validate2,to_categorical(y_validate2)))
            
            print('\n categorical crossentropy: ', end='')
            
            
        elif jj==2:
            ## categorical hinge
            # simple baseline model (to beat)
            msimple = Sequential([
                Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu',input_shape=(28,28,1)),
                Conv2D(64,(3,3),strides=(2,2),padding='same',activation='relu'),
                Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu'),
                Flatten(),
                Dense(256, activation='relu'),
                Dropout(.3),
                Dense(128, activation='relu'),
                Dropout(.3),
                Dense(10,activation='softmax')
            ])
            
            # compile model
            msimple.compile(optimizer='adam',
                            loss='categorical_hinge',
                            metrics=['accuracy'])
            
            # train model
            wsimple = msimple.fit(x_tr,to_categorical(y_tr),epochs=n_epoch,verbose=0,
                                  batch_size=64,
                                  validation_data=(x_validate2,to_categorical(y_validate2)))
            
            print('\n categorical hinge: ',end='')
            
        # save model acc
        tmp1 = wsimple.history['accuracy']
        acc[:,jj] += np.copy(tmp1) /n_survey
        acc_sq[:,jj] += np.copy(tmp1)**2 /n_survey
        tmp2 = wsimple.history['val_accuracy']
        vacc[:,jj] += np.copy(tmp2) /n_survey
        vacc_sq[:,jj] += np.copy(tmp2)**2 /n_survey
        print('acc ',np.round(np.max(tmp1),4),', val acc: ',np.round(np.max(tmp2),4))
    

In [ ]:
acc_std_err = (acc_sq - acc**2)**.5
vacc_std_err = (vacc_sq - vacc**2)**.5
models=['sparse categorical crossentropy','categorical crossentropy',
       'categorical hinge']
mcolor = ['r','b','y']

fig = plt.figure(figsize=(13,5))
plt.subplot(1,2,1)
plt.title('training accuracy')
for jj in range(n_models):
    plt.plot(np.arange(1,5),acc[:,jj],mcolor[jj]+'-', label=models[jj]);
    plt.plot(np.arange(1,5),acc[:,jj]+acc_std_err[:,jj],mcolor[jj]+'o');
    plt.plot(np.arange(1,5),acc[:,jj]-acc_std_err[:,jj],mcolor[jj]+'o');
    plt.legend(loc='lower right');
    plt.xlabel('number of epochs');

plt.subplot(1,2,2)
plt.title('validation accuracy')
for jj in range(n_models):
    plt.plot(np.arange(1,5),vacc[:,jj],mcolor[jj]+'-', label=models[jj]);
    plt.plot(np.arange(1,5),vacc[:,jj]+vacc_std_err[:,jj],mcolor[jj]+'o');
    plt.plot(np.arange(1,5),vacc[:,jj]-vacc_std_err[:,jj],mcolor[jj]+'o');
    plt.legend(loc='lower right');
    plt.xlabel('number of epochs');

Solid lines in the plots indicate the mean over `n_survey` (=6) rounds. The dots are the standard errors. We conclude, that the `categorical_entropy` and `sparse_categorical_crossentropy` are the loss functions we want to consider further. As it turned out, the loss function below it the best for our advanced model. 

In [16]:
our_loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#### Activation Functions
  <span style="color:red">*Explanation*</span>.

In [17]:
model = Sequential([
        # Normalize values from [0,255) to [-1, 1)
        Normalization(input_shape=x_train.shape[1:]),
    
        #3 convolutional layers followed by a max pooling layer
        Conv2D(32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        Conv2D(32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        Conv2D(32, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        #next 3 convolutional layers followed by max pooling layer
        Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        Conv2D(64, kernel_size=3, activation='relu', padding='same', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
    
        #3 dense layers followed by output layer
        Flatten(),
        Dense(256, activation='relu', kernel_regularizer=l2(L2_PENALTY)),
        BatchNormalization(),
        Dense(256, activation='relu', kernel_regularizer=l2(L2_PENALTY)),
        Dropout(0.25),

        Dense(256, activation='relu', kernel_regularizer=l2(L2_PENALTY)),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        1

## Kompilieren des Modells

In [19]:
model.compile(optimizer = our_optimizer,
                  loss= our_loss_function,
                  metrics = ['accuracy'])

## Training

In [20]:
# create data set using guide here https://medium.com/swlh/dump-keras-imagedatagenerator-start-using-tensorflow-tf-data-part-1-a30330bdbca9
# TODO double up training set and have over all 50% probability of augmentation happening at all?
# Set up training dataset
training_dataset = Dataset.from_tensor_slices((x_train2, y_train2))
training_dataset = training_dataset.map(dataset_aumentation, num_parallel_calls=tf.data.experimental.AUTOTUNE)
training_dataset = training_dataset.cache(TRAINING_CACHE_FILE)
training_dataset = training_dataset.shuffle(buffer_size=100)  # TODO set buffer size
training_dataset = training_dataset.repeat(1)  # TODO number
training_dataset = training_dataset.batch(BATCH_SIZE)
training_dataset = training_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Set up validation dataset
validation_dataset = Dataset.from_tensor_slices((x_validate2, y_validate2))
validation_dataset = validation_dataset.map(dataset_aumentation, num_parallel_calls=tf.data.experimental.AUTOTUNE)
validation_dataset = validation_dataset.cache(VALIDATION_CACHE_FILE)
validation_dataset = validation_dataset.shuffle(buffer_size=100)  # TODO set buffer size
validation_dataset = validation_dataset.repeat(1)  # TODO number
validation_dataset = validation_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
history = model.fit(
    training_dataset,
    epochs=NUM_EPOCHS,
    #steps_per_epoch=len(x_train2)/ BATCH_SIZE,
    verbose=1,
    validation_data=validation_dataset,
    #validation_steps=len(x_validate2)/ BATCH_SIZE,
    )

Epoch 1/200
750/750 [==============================] - 70s 87ms/step - loss: 4.8900 - accuracy: 0.5441 - val_loss: 3.7967 - val_accuracy: 0.8371
Epoch 2/200
750/750 [==============================] - 59s 79ms/step - loss: 3.7887 - accuracy: 0.8210 - val_loss: 3.4288 - val_accuracy: 0.8731
Epoch 3/200
410/750 [===============>..............] - ETA: 25s - loss: 3.4458 - accuracy: 0.8560

## Prediction

We use a custom algorithm for image prediction.
### Grid Search
When predicting, the image is augmented in multiple ways before forwaring it to the model. Each of these augmented images is evaluated by the model. Afterwards the mean of the resulting categories is calculated and used as actual prediction.

### Impact
This method results in a more stable prediction.  <span style="color:red">*TODO BEWEISE + Mehr schreiben*</span>.

In [ ]:
def predict(input_data: np.ndarray) -> np.ndarray:
    assert(input_data[0].shape == (28,28,1))
    augmentedResults = []
    
    augmentedResults.append(model.predict(input_data))
    
    flipped_images_vertical, flipped_images_horizontal, rot_images_90, rot_images_270 = augment_images(input_data)     
    augmentedResults.append(model.predict(flipped_images_vertical))
    augmentedResults.append(model.predict(flipped_images_horizontal))
    augmentedResults.append(model.predict(rot_images_90))
    augmentedResults.append(model.predict(rot_images_270))
    
    return (np.sum(augmentedResults, axis=0) / len(augmentedResults))

# Returns the accuracy
def evaluate(input_data: np.ndarray, input_labels: np.ndarray) -> float:
    assert(len(input_data) == len(input_labels))
    prediction = predict(input_data)
    prediction = np.argmax(prediction, axis = 1)
    
    return np.count_nonzero(prediction == input_labels) / len(input_labels)

## Visualization

In [ ]:
score_non_grid = model.evaluate(x_validate2,y_validate2,verbose=0)
score_grid = evaluate(x_validate2,y_validate2)
print('Test Accuracy without grid: {:.4f}'.format(score_non_grid[1]))
print('Test Accuracy with grid: {:.4f}'.format(score_grid))

In [ ]:
if SHOW_EXPLANATORY_GRAPHS:
    # Training Loss
    plt.figure(figsize=(20, 10))

    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Training - Loss Function')

In [ ]:
if SHOW_EXPLANATORY_GRAPHS:
    # Training Accuracy
    plt.figure(figsize=(20, 10))

    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Training - Accuracy Function')

In [ ]:
if SHOW_EXPLANATORY_GRAPHS:
    # Confusion matrix with grid
    val_pred = np.argmax(predict(x_validate2), axis = 1)

    conf_matrix = pd.DataFrame(confusion_matrix(y_validate2, val_pred), index=["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker","Bag", "Ankle boot"])
    plt.figure(figsize = (10,7))
    sns.heatmap(conf_matrix, annot=True)

In [ ]:
if SHOW_EXPLANATORY_GRAPHS:
    # Confusion matrix without grid
    val_pred = np.argmax(model.predict(x_validate2), axis = 1)

    conf_matrix = pd.DataFrame(confusion_matrix(y_validate2, val_pred), index=["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker","Bag", "Ankle boot"])
    plt.figure(figsize = (10,7))
    sns.heatmap(conf_matrix, annot=True)

## Submission

In [ ]:
# predict results
results = model.predict(data_submission)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
data_results = pd.DataFrame(results)
data_results.to_csv('fashion_mnist_pred_team1.csv', index=False)#Bitte statt X eure Gruppennummer einfügen! 

## Our Learnings
In the following part we reflect on our network and mention multiple points that could be improved for better classification performance.

### Parameters
Currently the hyperparameter tuning is done manually, but there are libraries (Like [this one](https://keras-team.github.io/keras-tuner/)) that implement systematic hyperparameter tuning. Since this process takes long and is very expensive (each setting needs to be tested after each other), and we (sadly) do not have the computational power to execute the process, we decided against using it. 


### Network Structure
In order to keep the computational complexity as low as possible we decided to use a more shallow structure.
Even though it performs very well research as well as the [official zalando github](https://github.com/zalandoresearch/fashion-mnist) suggest that a deeper network topography contributes to a more accurate classificaion.  [TODO](https://medium.com/@mjbhobe/classifying-fashion-with-a-keras-cnn-achieving-94-accuracy-part-3-c7ca2919232b)

Additionally other structures like DenseNet (described in [this paper](https://arxiv.org/abs/1608.06993)) or ResNet variations can be used to boost accuracy. Tensorflow offers a wide range of pretrained models that can be easily used ([list here](https://www.tensorflow.org/api_docs/python/tf/keras/applications)).

Also using cross validation (as described [here](https://towardsdatascience.com/3-methods-to-reduce-overfitting-of-machine-learning-models-4a7e2c1da9ef) and [here](https://www.machinecurve.com/index.php/2020/02/18/how-to-use-k-fold-cross-validation-with-keras/)) can help with minimizing overfitting, but it is computationally rather expensive since multiple models have to be trained.

### Data Preprocessing
While we used rudimentary data preprocessing functions, the model can benefit from a more extensive use of these methods. [This library](https://github.com/mdbloice/Augmentor) implements various image preprocessing functions that are well-suited for this purpose. Nethertheless the currently implemented preprocessor already increased the model's capability of generalization. A downside is that the accuracy fluctautes heavily in the beginning and training takes more epochs.

### Errors
<span style="color:red">*TODO Text überarbeiten (overfitting ist Problem,wie gegen overfitting vorgehen?, 94% gut, mehr Daten würden helfen)*</span>
The confusion matrix shows, that the network displays very poor classification capabilities on some classes (like dresses and shirts) whereas it has no difficulty categorizing others (i.e. boots and bags). Sadly we were not able to eliminate these shortcomings. However one of the methods presented above might be to do so.

### Workarounds
Almost every code uses workarounds and this one is no exception. For example the workaround for using the random eraser could be exchanged with some effort (i.e. converting the random eraser to use a Tensor). This could result in a better runtime performance (since image generation is triggered by tensorflow and potentially calculated on the GPU) as well as the utilization of cache for less memory usage.